In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()
h2o.cluster().shutdown(prompt=True) 

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.144-b01, mixed mode)
  Starting server from C:\Users\rishi\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\rishi\AppData\Local\Temp\tmpocjvmnez
  JVM stdout: C:\Users\rishi\AppData\Local\Temp\tmpocjvmnez\h2o_rishi_started_from_python.out
  JVM stderr: C:\Users\rishi\AppData\Local\Temp\tmpocjvmnez\h2o_rishi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_rishi_bz7iqy
H2O cluster total nodes:,1
H2O cluster free memory:,3.528 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Are you sure you want to shutdown the H2O instance running at http://127.0.0.1:54321 (Y/N)? N


In [5]:
df = h2o.import_file('./historical_data1_time_Q12005.txt')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
df.columns=['loan_number','year-month', 'current_actual_upb', 'delq_status','loan_age', 'rem_months', 'repurchase_flag', 'modification_flag', 'zero_balance_code',
                    'zero_bal_date', 'org_int_rate', 'current_def_upb', 'ddlpi', 'mi_recoveries', 'net_sales_proceeds',
                    'non_mi_recoveries', 'expenses', 'legal_costs', 'maint_pres_costs', 'taxes_ins', 'misc_expenses',
                    'actual_loss_calc', 'modification_cost', 'stepmod_ind' , 'dpm_ind' , 'eltv' ]


In [14]:
def statusDeliquent(df):
        if df['delq_status'] > 0:
            val = 1
        else:
            val = 0
        return val 

In [24]:
df.apply( fun = lambda x : x.statusDeliquent(), axis =1)

ValueError: unimpl bytecode instr: CALL_METHOD

In [22]:
y = 'delq_status'

In [72]:
aml = H2OAutoML(max_runtime_secs = 6000, seed = 1)

In [73]:
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

In [74]:
#import numpy as np

In [75]:
h2oServer = h2o.init(nthreads = -1, max_mem_size = "7G")

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 min 13 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_Komal_f7xzqs
H2O cluster total nodes:,1
H2O cluster free memory:,2.723 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [ ]:
aml.train(y = y, training_frame = train, leaderboard_frame =test )

AutoML progress: |████████████████████████████████████

In [49]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20181128_184249,0.0589145,0.242723,0.0589145,0.179068,0.0362195
StackedEnsemble_BestOfFamily_AutoML_20181128_184249,0.0594723,0.243869,0.0594723,0.179735,0.0363882
GBM_4_AutoML_20181128_184249,0.0597037,0.244343,0.0597037,0.180205,0.0364556
GBM_5_AutoML_20181128_184249,0.0598676,0.244679,0.0598676,0.180461,0.0365096
GBM_3_AutoML_20181128_184249,0.0600509,0.245053,0.0600509,0.180747,0.0365571
GBM_2_AutoML_20181128_184249,0.0602627,0.245485,0.0602627,0.181252,0.0366201
GBM_grid_1_AutoML_20181128_184249_model_6,0.0602709,0.245501,0.0602709,0.181097,0.0366342
GBM_grid_1_AutoML_20181128_184249_model_5,0.0602904,0.245541,0.0602904,0.181585,0.0366414
GBM_1_AutoML_20181128_184249,0.060748,0.246471,0.060748,0.182013,0.0367763
GBM_grid_1_AutoML_20181128_184249_model_1,0.0619842,0.248966,0.0619842,0.184051,0.0371497


In [50]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.05891454988282391
RMSE: 0.2427231960131209
MAE: 0.17906808363506316
RMSLE: 0.036219535047174754
R^2: 0.5603185758078277
Mean Residual Deviance: 0.05891454988282391
Null degrees of freedom: 70133
Residual degrees of freedom: 70123
Null deviance: 9398.106447521634
Residual deviance: 4131.913041481972
AIC: 459.5233347959999
